In [ ]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" sentencepiece fire einops --upgrade

In [ ]:
def format_instruction(sample):
	return f"""### Instruction:
Responda à pergunta com a maior sinceridade possível usando o CONTEXTO fornecido e, se a resposta não estiver contida no CONTEXTO abaixo, diga 'Desculpe, não possuo essa informação'.

### Input:
CONTEXTO: {sample['context']}

PERGUNTA: {sample['question']}

### Response: {sample['resposta']}
"""

In [ ]:
from datasets import load_dataset
from random import randrange

dataset = load_dataset("Weni/LLM-base", split='train')
dataset = dataset.shuffle(seed=55)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "upstage/Llama-2-70b-instruct-v2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="auto", use_safetensors=False)
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="upstage-70",
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=6e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset.with_format("torch"),
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)

In [ ]:
trainer.train()

trainer.save_model()

In [ ]:
output_dir = 'upstage-70-files'
base_model_name_or_path = 'upstage/Llama-2-70b-instruct-v2'
device_arg = { 'device_map': 'auto' }

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    return_dict=True,
    torch_dtype=torch.float16,
    **device_arg
)

model = PeftModel.from_pretrained(base_model, 'upstage-70', **device_arg)
model = model.merge_and_unload()

tokenizer.save_pretrained(output_dir)
model.save_pretrained(output_dir)